In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from transformers import BertModel, BertTokenizer

# Implemented by myself
from config import *
from data_processer import CSCDataset, split_torch_dataset
from models import CombineBertModel, DecoderBaseRNN, DecoderTransformer

/root/anaconda3/envs/simple_csc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from utils import cal_err

# def cal_err(raw_sentence, pred_sentence, corr_sentence, limit_length=300):
#     matrices = ["over_corr", "total_err", "true_corr"]
#     char_level = {key: 0 for key in matrices}
#     sent_level = {key: 0 for key in matrices}

#     f1 = f2 = 0
#     for i, c in enumerate(raw_sentence):
#         if i >= limit_length:
#             break

#         pc, cc = pred_sentence[i], corr_sentence[i]

#         if cc != c:
#             char_level["total_err"] += 1
#             char_level["true_corr"] += pc == cc
#             f1 = 1
#         elif pc != cc:
#             char_level["over_corr"] += 1
#             f2 = 1

#     if f1:
#         sent_level["true_corr"] += all(pred_sentence == corr_sentence)
#         sent_level["total_err"] += f1
#     sent_level["over_corr"] += f2

#     return char_level, sent_level

In [3]:
tokenizer = BertTokenizer.from_pretrained(checkpoint)

In [4]:
# def split_torch_dataset(dataset, test_size=0.2, random_seed=None):
#     """
#     将PyTorch Dataset对象划分为训练集和测试集。

#     参数:
#         dataset (Dataset): PyTorch的Dataset对象。
#         test_size (float or int): 如果是浮点数，表示测试集相对于整个数据集的比例；如果是整数，表示测试集的样本数量。默认值为0.2。
#         random_seed (int or None): 随机种子，用于控制随机性以确保结果可复现。默认值为None。

#     返回:
#         tuple: 一个包含两个Dataset对象的元组 (train_dataset, test_dataset)。
#     """
#     # 计算训练集和测试集的大小
#     total_length = len(dataset)
#     if isinstance(test_size, float):
#         test_length = int(total_length * test_size)
#     else:
#         test_length = test_size
#     train_length = total_length - test_length

#     # 设置随机种子
#     generator = (
#         torch.Generator().manual_seed(random_seed) if random_seed is not None else None
#     )

#     # 分割数据集
#     train_dataset, test_dataset = random_split(
#         dataset, [train_length, test_length], generator=generator
#     )

#     return train_dataset, test_dataset

In [5]:
train_dataset = CSCDataset([SIGHAN_train_dir_err, SIGHAN_train_dir_corr], tokenizer)

test_dataset = CSCDataset([SIGHAN_train_dir_err14, SIGHAN_train_dir_corr14], tokenizer)

preprocessing sighan dataset: 2339it [00:00, 463479.81it/s]
preprocessing sighan dataset: 100%|██████████| 2339/2339 [00:00<00:00, 812529.16it/s]


共2339句，共73264字，最长的句子有171字


preprocessing sighan dataset: 3437it [00:00, 545847.14it/s]
preprocessing sighan dataset: 100%|██████████| 3437/3437 [00:00<00:00, 825601.22it/s]

共3437句，共170330字，最长的句子有258字


In [6]:
train_data, dev_data = split_torch_dataset(train_dataset, 0.3)

train_data_loader = DataLoader(train_data, num_workers=4, shuffle=True, batch_size=16)

dev_data_loader = DataLoader(dev_data, num_workers=4, shuffle=True, batch_size=16)

test_data_loader = DataLoader(test_dataset, num_workers=4, shuffle=True, batch_size=32)

In [7]:
def train(model, tokenizer, train_data_loader, test_data_loader=None):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(
            enumerate(train_data_loader),
            desc=f"Epoch:{epoch+1}/{epochs}",
            total=len(train_data_loader),
        )

        for i, batch in progress_bar:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

            # 反向传播在这，故labels不需要传入模型
            loss = F.cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

        # if epoch % 5 == 0:
        with torch.no_grad():
            t = torch.argmax(outputs, dim=-1)
            nt = t * attention_mask
            pred = tokenizer.batch_decode(nt, skip_special_tokens=True)
        # print(pred)
        # print(f"origin{tokenizer.batch_decode(labels, skip_special_tokens=True)}")

        for i, v in enumerate(nt):
            r, l = input_ids[i], labels[i]
            limit_length = sum(attention_mask[i].to("cpu"))
            print(tokenizer.decode(r, skip_special_tokens=True))
            print(tokenizer.decode(v, skip_special_tokens=True))
            print(tokenizer.decode(l, skip_special_tokens=True))
            print(cal_err(r, v, l, limit_length))
            # print(cal_err(r, v, l))

        print(f"Epoch {epoch+1} Loss: {total_loss / len(train_data_loader)}")

        # dev
        if test_data_loader:
            test(model, tokenizer, test_data_loader)

        # 释放不必要的内存
        # del input_ids, attention_mask, labels, outputs, loss
        # torch.cuda.empty_cache()
        # gc.collect()

In [8]:
def test(model, tokenizer, test_data_loader):
    model.eval()
    total_loss = 0
    matrices = ["over_corr", "total_err", "true_corr"]
    test_char_level = {key: 0 for key in matrices}
    test_sent_level = {key: 0 for key in matrices}
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    with torch.no_grad():
        for batch in test_data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)

            loss = F.cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()

            t = torch.argmax(outputs, dim=-1)
            nt = t * attention_mask
            pred = tokenizer.batch_decode(nt, skip_special_tokens=True)

            for i in range(len(t)):
                char_level, sent_level = cal_err(
                    input_ids[i], nt[i], labels[i], sum(attention_mask[i].to("cpu"))
                )
                test_char_level = {
                    key: test_char_level[key] + v for key, v in char_level.items()
                }
                test_sent_level = {
                    key: test_sent_level[key] + v for key, v in sent_level.items()
                }
        print(total_loss / len(test_data_loader), test_char_level, test_sent_level)

#### BERT + LSTM
**长度不一致问题**：预测的时候有些字符预测为了 **[SEP]**

In [9]:
epochs = 35

# tokenizer = BertTokenizer.from_pretrained(checkpoint)
encoder_model = BertModel.from_pretrained(checkpoint)

# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

decoder_model = DecoderBaseRNN(
    model=nn.LSTM,
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)

model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

train(model, tokenizer, train_data_loader, dev_data_loader)

Epoch:1/35: 100%|██████████| 103/103 [00:28<00:00,  3.61it/s, loss=5.904]


我 覺 得 安 裝 攝 影 機 會 影 響 到 孩 子 的 成 長 ， 並 對 家 長 的 撫 育 觀 念 不 利 。 最 後 ， 攝 影 機 可 能 因 起 隱 私 權 的 法 律 問 題 。
我 的 的 的 的 的 ， ， ， ， ， ， ， ， 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的
我 覺 得 安 裝 攝 影 機 會 影 響 到 孩 子 的 成 長 ， 並 對 家 長 的 撫 育 觀 念 不 利 。 最 後 ， 攝 影 機 可 能 引 起 隱 私 權 的 法 律 問 題 。
({'over_corr': 47, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
學 校 把 學 生 交 給 老 師 教 ， 小 長 應 該 信 任 老 師 。
我 的 的 的 的 的 ， ， ， ， ， ， ， ， 的 的 的 的 的 的
學 校 把 學 生 交 給 老 師 教 ， 校 長 應 該 信 任 老 師 。
({'over_corr': 19, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
今 天 給 您 寫 了 一 封 信 ， 希 望 您 趕 快 找 處 理 讓 我 們 可 以 睡 個 飽 覺 。
我 的 的 的 的 的 ， ， ， ， ， ， ， ， ， 的 的 的 的 的 的 的 的 的 的 的 的 的
今 天 給 您 寫 了 一 封 信 ， 希 望 您 趕 快 早 處 理 讓 我 們 可 以 睡 個 飽 覺 。
({'over_corr': 27, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
簡 單 的 說 ， 你 要 運 用 的 個 時 間 來 投 資 你 自 己

Epoch:2/35: 100%|██████████| 103/103 [00:28<00:00,  3.68it/s, loss=6.046]


去 年 ， 美 國 華 人 友 請 我 們 來 桃 園 看 叔 叔 家 。
我 我 ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，
去 年 ， 美 國 華 人 有 請 我 們 來 桃 園 看 叔 叔 家 。
({'over_corr': 19, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
簡 單 來 講 ， 先 馬 鈴 薯 ， 洋 蔥 ， 紅 蘿 蔔 炒 以 炒 ， 加 湯 汁 ， 牛 肉 ， 小 匙 糖 ， 醬 油 ， 甜 料 酒 ， 煮 到 馬 鈴 薯 軟 就 可 以 。
我 我 ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，
簡 單 來 講 ， 先 馬 鈴 薯 ， 洋 蔥 ， 紅 蘿 蔔 炒 一 炒 ， 加 湯 汁 ， 牛 肉 ， 小 匙 糖 ， 醬 油 ， 甜 料 酒 ， 煮 到 馬 鈴 薯 軟 就 可 以 。
({'over_corr': 39, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
春 天 的 時 候 開 花 ， 什 麼 地 方 都 錄 了 。
我 我 ， ， ， ， ， ， ， ， ， ， ， ， ， ，
春 天 的 時 候 開 花 ， 什 麼 地 方 都 綠 了 。
({'over_corr': 15, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
開 車 的 人 聽 了 他 的 華 馬 上 說 ： 「 我 可 以 用 我 的 車 陪 你 去 ， 好 不 好 ？ 」
我 我 ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，
開

Epoch:3/35: 100%|██████████| 103/103 [00:28<00:00,  3.65it/s, loss=5.804]


...

可 能 政 府 會 決 定 你 們 可 以 般 到 別 的 地 方 ， 還 有 可 能 會 給 你 們 更 多 錢 。
可 能 政 府 會 決 定 你 們 可 以 搬 到 別 的 地 方 ， 還 有 可 能 會 給 你 們 更 多 錢 。
可 能 政 府 會 決 定 你 們 可 以 搬 到 別 的 地 方 ， 還 有 可 能 會 給 你 們 更 多 錢 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
我 覺 得 校 長 的 作 法 對 家 長 、 孩 子 跟 學 校 都 不 好 。
我 覺 得 校 長 的 做 法 對 家 長 、 孩 子 跟 學 校 都 不 好 。
我 覺 得 校 長 的 做 法 對 家 長 、 孩 子 跟 學 校 都 不 好 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
作 天 我 看 到 你 太 太 ， 她 告 訴 你 有 感 冒 。
昨 天 我 看 到 你 太 太 ， 她 告 訴 你 有 感 冒 。
昨 天 我 看 到 你 太 太 ， 她 告 訴 你 有 感 冒 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
我 覺 得 他 票 亮 得 很 。
我 覺 得 他 漂 亮 得 很 。
我 覺 得 他 漂 亮 得 很 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
救 我 的 想 法 呢 ， 我 可 能 也 會 很 難 過 吧 但 

Epoch:34/35: 100%|██████████| 103/103 [00:28<00:00,  3.66it/s, loss=0.171]


我 希 望 你 的 病 越 來 越 好 ， 身 體 越 來 越 建 康 。
我 希 望 你 的 病 越 來 越 好 ， 身 體 越 來 越 健 康 。
我 希 望 你 的 病 越 來 越 好 ， 身 體 越 來 越 健 康 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
第 三 個 象 片 兒 ： 兩 個 人 都 坐 公 共 汽 車 還 是 昨 坐 火 車 。
第 三 個 相 片 兒 ： 兩 個 人 都 坐 公 共 汽 車 還 是 昨 坐 火 車 。
第 三 個 相 片 兒 ： 兩 個 人 都 坐 公 共 汽 車 還 是 昨 坐 火 車 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
我 會 支 持 妳 呀 ， 加 油 喔 ！ 我 也 偷 偷 的 跟 大 年 溝 通 一 下 吧 。
我 會 支 持 妳 呀 ， 加 油 喔 ！ 我 也 偷 偷 地 跟 大 年 溝 通 一 下 吧 。
我 會 支 持 妳 呀 ， 加 油 喔 ！ 我 也 偷 偷 地 跟 大 年 溝 通 一 下 吧 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
我 的 生 日 那 天 ， 我 沒 有 邀 請 太 多 的 人 。 我 們 可 以 開 心 的 慶 祝 好 之 後 ， 就 一 起 來 規 劃 我 們 的 未 來 。
我 的 生 日 那 天 ， 我 沒 有 邀 請 太 多 的 人 。 我 們 可 以 開 心 地 慶 祝 好 之 後 ， 就 一 起 來 規 劃 我 們 的 未 來 。
我 的 生 日 那 天 ， 我 沒 有 邀 請 太 多 的 人 。 我 們 可 以 開 心 地 慶 祝 

Epoch:35/35: 100%|██████████| 103/103 [00:28<00:00,  3.64it/s, loss=0.168]


畢 竟 ， 如 果 你 跟 你 的 這 個 朋 友 再 提 出 這 個 問 題 ， 說 不 定 會 邊 得 更 嚴 重 。
畢 竟 ， 如 果 你 跟 你 的 這 個 朋 友 再 提 出 這 個 問 題 ， 說 不 定 會 變 得 更 嚴 重 。
畢 竟 ， 如 果 你 跟 你 的 這 個 朋 友 再 提 出 這 個 問 題 ， 說 不 定 會 變 得 更 嚴 重 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
部 是 指 人 類 有 心 或 感 情 ， 它 們 也 會 感 到 歡 樂 、 也 會 感 到 悲 哀 。
不 是 只 人 類 有 心 或 感 情 ， 它 們 也 會 感 到 歡 樂 、 也 會 感 到 悲 塞 。
不 是 只 人 類 有 心 或 感 情 ， 它 們 也 會 感 到 歡 樂 、 也 會 感 到 悲 哀 。
({'over_corr': 1, 'total_err': 2, 'true_corr': tensor(2, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
他 們 只 能 有 兩 個 選 擇 ： 接 受 降 新 或 自 動 離 職 。
他 們 只 能 有 兩 個 選 擇 ： 接 受 減 薪 或 自 動 離 職 。
他 們 只 能 有 兩 個 選 擇 ： 接 受 降 薪 或 自 動 離 職 。
({'over_corr': 1, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
但 還 是 希 望 你 首 先 平 息 你 的 反 應 ， 其 次 儘 量 跟 那 位 同 學 購 通 你 們 倆 之 間 的 問 題 及 對 這 件 事 情 的 看 法 ， 好 讓 不 要 再 發 生 這 樣 傷 害 彼 此 的 事 情 。
但 還 是 希 望 你 首 先 平 息 你 的 反 應 ， 其 

In [10]:
test(model, tokenizer, test_data_loader)

0.5369593727367895 {'over_corr': 4982, 'total_err': 5278, 'true_corr': tensor(1425, device='cuda:0')} {'over_corr': 1999, 'total_err': 3436, 'true_corr': 381}


#### BERT + GRU

In [11]:
encoder_model = BertModel.from_pretrained(checkpoint)

# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

decoder_model = DecoderBaseRNN(
    model=nn.GRU,
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)

model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

train(model, tokenizer, train_data_loader, dev_data_loader)

Epoch:1/35: 100%|██████████| 103/103 [00:27<00:00,  3.78it/s, loss=5.318]


王 天 華 開 心 得 一 直 說 話 。
我 我 ， ， ， ， ， ， ， ，
王 天 華 開 心 地 一 直 說 話 。
({'over_corr': 10, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
上 週 四 早 上 ， 我 一 如 往 常 的 至 住 家 附 近 的 頂 好 超 市 買 菜 。
我 我 ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，
上 週 四 早 上 ， 我 一 如 往 常 地 至 住 家 附 近 的 頂 好 超 市 買 菜 。
({'over_corr': 23, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
也 許 老 師 可 以 把 錄 音 給 父 母 看 ， 所 以 他 們 知 道 他 們 的 孩 子 這 麼 樣 ， 也 是 這 麼 樣 教 他 們 的 孩 子 。
我 我 ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，
也 許 老 師 可 以 把 錄 影 給 父 母 看 ， 所 以 他 們 知 道 他 們 的 孩 子 怎 麼 樣 ， 也 是 怎 麼 樣 教 他 們 的 孩 子 。
({'over_corr': 36, 'total_err': 3, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
所 以 你 好 好 得 想 要 怎 麼 做 ， 如 果 要 我 幫 甚 麼 忙 隨 時 都 可 以 告 訴 我 。
我 我 ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，
所 以 你 好 好 地 想 要 怎 麼 做 ， 如 果 要 我 幫 甚 麼 忙 隨 時 都 可 以

Epoch:2/35: 100%|██████████| 103/103 [00:27<00:00,  3.76it/s, loss=4.695]


但 我 們 的 目 的 還 是 相 同 的 ， 也 是 為 了 家 庭 根 社 會 做 貢 獻 ， 而 只 是 得 到 目 的 的 方 法 不 一 樣 。
一 我 們 的 的 的 ， 我 有 一 的 ， ， 我 一 的 的 一 一 ， 的 ， ， 不 ， 一 的 一 的 的 的 的 不 一 一 。
但 我 們 的 目 的 還 是 相 同 的 ， 也 是 為 了 家 庭 跟 社 會 做 貢 獻 ， 而 只 是 得 到 目 的 的 方 法 不 一 樣 。
({'over_corr': 26, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
我 門 可 以 到 外 面 ， 吃 很 多 東 西 了 。
我 們 ， 們 有 有 的 ， 的 我 ， 一 一 的 。
我 們 可 以 到 外 面 ， 吃 很 多 東 西 了 。
({'over_corr': 11, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
我 希 望 你 神 病 快 好 ， 你 要 吃 多 水 果 ， 以 後 我 們 可 以 一 起 吃 飯 。
我 一 一 你 一 的 一 ， ， 我 們 的 ， ， ， 們 ， 我 們 ， ， 一 一 的 一 。
我 希 望 你 生 病 快 好 ， 你 要 吃 多 水 果 ， 以 後 我 們 可 以 一 起 吃 飯 。
({'over_corr': 18, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
他 從 北 投 捷 運 站 坐 捷 運 到 估 停 站 。
他 們 一 一 的 的 一 的 的 。
他 從 北 投 捷 運 站 坐 捷 運 到 古 亭 站 。
({'over_corr': 11, 'total_err': 2, 'true_corr': tensor(0, dev

Epoch:3/35: 100%|██████████| 103/103 [00:27<00:00,  3.78it/s, loss=2.668]


...

反 而 ， 只 好 讓 他 們 每 天 上 網 已 定 的 時 間 。 要 不 然 就 有 很 嚴 格 的 制 裁 。
反 而 ， 只 好 讓 他 們 每 天 上 網 一 定 的 時 間 。 要 不 然 就 有 很 嚴 格 的 制 裁 。
反 而 ， 只 好 讓 他 們 每 天 上 網 一 定 的 時 間 。 要 不 然 就 有 很 嚴 格 的 制 裁 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
作 天 ， 我 爬 山 ， 空 氣 好 還 有 看 法 很 漂 亮 。
昨 天 ， 我 爬 山 ， 空 氣 好 還 有 看 法 很 漂 亮 。
昨 天 ， 我 爬 山 ， 空 氣 好 還 有 看 法 很 漂 亮 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
攝 影 機 的 特 色 是 可 以 看 到 小 孩 在 教 室 裡 做 什 麼 ， 家 長 可 以 知 道 自 己 孩 子 上 課 的 態 度 這 麼 樣 ， 像 又 沒 有 主 動 參 加 教 室 裡 的 活 動 ， 每 天 學 習 什 麼 等 等 。
攝 影 機 的 特 色 是 可 以 看 到 小 孩 在 教 室 裡 做 什 麼 ， 家 長 可 以 知 道 自 己 孩 子 上 課 的 態 度 怎 麼 樣 ， 像 有 沒 有 主 動 參 加 教 室 裡 的 活 動 ， 每 天 學 習 什 麼 等 等 。
攝 影 機 的 特 色 是 可 以 看 到 小 孩 在 教 室 裡 做 什 麼 ， 家 長 可 以 知 道 自 己 孩 子 上 課 的 態 度 怎 麼 樣 ， 像 有 沒 有 主 動 參 加 教 室 裡 的 活 動 ， 每 天 學 習 什 麼 等 等 。
({'over_corr': 0, 'total_err': 2, 'true_corr': tensor(2, device='cuda:0')}, 

Epoch:34/35: 100%|██████████| 103/103 [00:27<00:00,  3.78it/s, loss=0.005]


我 看 他 的 時 候 我 很 高 心 ， 因 為 上 子 我 看 他 是 一 年 以 前 。
我 看 他 的 時 候 我 很 高 興 ， 因 為 上 次 我 看 他 是 一 年 以 前 。
我 看 他 的 時 候 我 很 高 興 ， 因 為 上 次 我 看 他 是 一 年 以 前 。
({'over_corr': 0, 'total_err': 2, 'true_corr': tensor(2, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
我 的 弟 弟 也 很 高 薪 看 到 你 們 ， 因 為 你 們 好 久 不 見 沒 看 到 。
我 的 弟 弟 也 很 高 興 看 到 你 們 ， 因 為 你 們 好 久 不 見 沒 看 到 。
我 的 弟 弟 也 很 高 興 看 到 你 們 ， 因 為 你 們 好 久 不 見 沒 看 到 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
在 說 ， 我 覺 得 這 裡 也 可 以 有 很 好 玩 ， 也 可 以 學 很 多 很 好 的 東 西 。
再 說 ， 我 覺 得 這 裡 也 可 以 有 很 好 玩 ， 也 可 以 學 很 多 很 好 的 東 西 。
再 說 ， 我 覺 得 這 裡 也 可 以 有 很 好 玩 ， 也 可 以 學 很 多 很 好 的 東 西 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
今 天 早 藏 ， 我 們 八 點 出 公 車 贊 。
今 天 早 上 ， 我 們 八 點 出 公 車 站 。
今 天 早 上 ， 我 們 八 點 出 公 車 站 。
({'over_corr': 0, 'total_err': 2, 'true_corr': tensor(2, device='cuda:0')}, 

Epoch:35/35: 100%|██████████| 103/103 [00:27<00:00,  3.78it/s, loss=0.009]


雖 然 筆 找 到 了 也 證 明 我 的 清 白 但 整 件 事 情 會 讓 我 心 理 遭 到 傷 痕 呀 ！ 所 以 我 很 瞭 解 你 現 在 的 心 情 。
雖 然 筆 找 到 了 也 證 明 我 的 清 白 但 整 件 事 情 會 讓 我 心 裡 遭 到 傷 痕 呀 ！ 所 以 我 很 瞭 解 你 現 在 的 心 情 。
雖 然 筆 找 到 了 也 證 明 我 的 清 白 但 整 件 事 情 會 讓 我 心 裡 遭 到 傷 痕 呀 ！ 所 以 我 很 瞭 解 你 現 在 的 心 情 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
開 車 的 人 聽 了 他 的 華 馬 上 說 ： 「 我 可 以 用 我 的 車 陪 你 去 ， 好 不 好 ？ 」
開 車 的 人 聽 了 他 的 話 馬 上 說 ： 「 我 可 以 用 我 的 車 陪 你 去 ， 好 不 好 ？ 」
開 車 的 人 聽 了 他 的 話 馬 上 說 ： 「 我 可 以 用 我 的 車 陪 你 去 ， 好 不 好 ？ 」
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
由 於 上 述 係 區 民 所 抱 怨 項 目 ， 我 們 在 此 敬 請 您 方 進 行 環 境 改 善 。
由 於 上 述 係 居 民 所 抱 怨 項 目 ， 我 們 在 此 敬 請 您 方 進 行 環 境 改 善 。
由 於 上 述 係 居 民 所 抱 怨 項 目 ， 我 們 在 此 敬 請 您 方 進 行 環 境 改 善 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
我 們 跳 無 完 的 以 後 ， 就 回 

In [12]:
test(model, tokenizer, test_data_loader)

0.45158677106654205 {'over_corr': 3414, 'total_err': 5278, 'true_corr': tensor(1152, device='cuda:0')} {'over_corr': 1435, 'total_err': 3436, 'true_corr': 421}


#### BERT + transformers

In [13]:
encoder_model = BertModel.from_pretrained(checkpoint)

nhead = 2
num_encoder_layers = 2
num_decoder_layers = 2

decoder_model = DecoderTransformer(
    input_size=encoder_model.config.hidden_size,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
)

model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

train(model, tokenizer, train_data_loader, dev_data_loader)

Epoch:1/35: 100%|██████████| 103/103 [00:26<00:00,  3.85it/s, loss=1.651]


還 有 ， 你 們 工 廠 可 以 使 用 檔 聲 音 的 玻 璃 。 那 麼 聲 音 與 臭 味 不 能 傳 到 外 面 ， 那 一 定 是 很 好 的 辦 法 。
還 有 ， 你 們 工 廠 可 以 使 用 我 聲 音 的 電 。 那 麼 聲 音 與 臭 味 不 能 到 外 面 ， 那 一 定 是 很 好 的 辦 法 。
還 有 ， 你 們 工 廠 可 以 使 用 擋 聲 音 的 玻 璃 。 那 麼 聲 音 與 臭 味 不 能 傳 到 外 面 ， 那 一 定 是 很 好 的 辦 法 。
({'over_corr': 3, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
今 天 大 明 第 一 次 到 十 大 去 ， 他 是 新 生 在 師 大 。
今 天 大 明 第 一 次 到 十 大 去 ， 他 是 新 生 在 師 大 。
今 天 大 明 第 一 次 到 師 大 去 ， 他 是 新 生 在 師 大 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
爸 爸 請 你 在 一 次 想 一 想 好 好 兒 的 辦 法 。
爸 爸 請 你 在 一 次 想 一 想 好 好 兒 的 辦 法 。
爸 爸 請 你 再 一 次 想 一 想 好 好 兒 的 辦 法 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
我 想 拿 出 來 一 些 啤 酒 ， 一 些 平 乾 。
我 想 拿 出 來 一 些 。 酒 ， 一 些 平 。 。
我 想 拿 出 來 一 些 啤 酒 ， 一 些 餅 乾 。
({'over_corr': 2, 'total_err': 1, 'true_corr': tensor(0, devic

Epoch:2/35: 100%|██████████| 103/103 [00:26<00:00,  3.84it/s, loss=1.020]


格 位 學 生 的 父 母 ， 您 門 好 。
回 位 學 生 的 父 母 ， 您 們 好 。
各 位 學 生 的 父 母 ， 您 們 好 。
({'over_corr': 0, 'total_err': 2, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
如 果 你 有 空 ， 我 想 這 個 禮 拜 五 情 你 去 吃 飯 。
如 果 你 有 空 ， 我 想 這 個 禮 拜 五 情 你 去 吃 飯 。
如 果 你 有 空 ， 我 想 這 個 禮 拜 五 請 你 去 吃 飯 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
所 以 我 比 較 想 推 薦 其 他 餐 廳 ， 就 是 一 本 導 游 書 裡 面 的 餐 廳 。
所 以 我 比 較 想 推 出 其 他 餐 廳 ， 就 是 一 本 導 遊 書 裡 面 的 餐 廳 。
所 以 我 比 較 想 推 薦 其 他 餐 廳 ， 就 是 一 本 導 遊 書 裡 面 的 餐 廳 。
({'over_corr': 1, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
對 我 來 說 要 是 我 受 傷 也 感 冒 也 都 沒 問 題 ， 我 總 是 一 定 上 課 。 可 是 如 果 我 的 血 綠 人 出 事 的 話 ， 我 不 能 上 課 。
對 我 來 說 要 是 我 受 傷 也 感 冒 也 都 沒 問 題 ， 我 總 是 一 定 上 課 。 可 是 如 果 我 的 希 現 人 出 事 的 話 ， 我 不 能 上 課 。
對 我 來 說 要 是 我 受 傷 也 感 冒 也 都 沒 問 題 ， 我 總 是 一 定 上 課 。 可 是 如 果 我 的 血 緣 人 出 事 的 話 ， 我 不 能 上 課 。
({'o

Epoch:3/35: 100%|██████████| 103/103 [00:26<00:00,  3.83it/s, loss=0.614]


...

看 教 室 的 情 況 讓 父 母 安 全 ， 他 們 知 道 他 們 的 孩 子 這 麼 樣 。
看 教 室 的 情 況 讓 父 母 安 全 ， 他 們 知 道 他 們 的 孩 子 怎 麼 樣 。
看 教 室 的 情 況 讓 父 母 安 全 ， 他 們 知 道 他 們 的 孩 子 怎 麼 樣 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
我 們 受 不 了 了 ！ 李 廠 廠 你 必 須 改 變 工 廠 ！ 我 要 告 訴 您 怎 麼 解 決 我 們 的 問 題 。
我 們 受 不 了 了 ！ 李 廠 長 你 必 須 改 變 工 廠 ！ 我 要 告 訴 您 怎 麼 解 決 我 們 的 問 題 。
我 們 受 不 了 了 ！ 李 廠 長 你 必 須 改 變 工 廠 ！ 我 要 告 訴 您 怎 麼 解 決 我 們 的 問 題 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
下 個 星 期 五 是 我 的 生 日 ， 所 以 我 要 開 一 個 無 會 。
下 個 星 期 五 是 我 的 生 日 ， 所 以 我 要 開 一 個 舞 會 。
下 個 星 期 五 是 我 的 生 日 ， 所 以 我 要 開 一 個 舞 會 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
爸 爸 剛 開 始 也 和 你 現 在 一 樣 ， 但 是 因 為 媽 媽 ， 家 人 的 鼓 勵 和 支 持 ， 爸 爸 還 是 努 力 的 找 尋 新 工 作 。
爸 爸 剛 開 始 也 和 你 現 在 一 樣 ， 但 是 因 為 媽 媽 ， 家 人 的 鼓 勵 和 支 持 ， 爸 爸 還 

Epoch:34/35: 100%|██████████| 103/103 [00:26<00:00,  3.83it/s, loss=0.002]


感 覺 到 無 可 奈 何 ， 不 知 道 怎 麼 辦 在 行 了 。
感 覺 到 無 可 奈 何 ， 不 知 道 怎 麼 辦 才 行 了 。
感 覺 到 無 可 奈 何 ， 不 知 道 怎 麼 辦 才 行 了 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
這 樣 也 可 以 幫 父 母 了 解 孩 子 的 件 況 ， 他 們 對 什 麼 還 有 問 題 ， 所 以 父 母 可 以 鼓 勵 孩 子 。
這 樣 也 可 以 幫 父 母 了 解 孩 子 的 情 況 ， 他 們 對 什 麼 還 有 問 題 ， 所 以 父 母 可 以 鼓 勵 孩 子 。
這 樣 也 可 以 幫 父 母 了 解 孩 子 的 情 況 ， 他 們 對 什 麼 還 有 問 題 ， 所 以 父 母 可 以 鼓 勵 孩 子 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
雖 然 你 也 自 己 說 ， 幸 虧 你 還 有 工 作 ， 所 以 請 你 每 周 兩 天 好 好 地 工 作 ， 為 了 提 高 自 己 的 能 力 ， 另 外 每 周 五 天 ， 請 你 好 好 地 學 習 自 己 有 興 趣 的 事 情 。
雖 然 你 也 自 己 說 ， 幸 虧 你 還 有 工 作 ， 所 以 請 你 每 週 兩 天 好 好 地 工 作 ， 為 了 提 高 自 己 的 能 力 ， 另 外 每 週 五 天 ， 請 你 好 好 地 學 習 自 己 有 興 趣 的 事 情 。
雖 然 你 也 自 己 說 ， 幸 虧 你 還 有 工 作 ， 所 以 請 你 每 週 兩 天 好 好 地 工 作 ， 為 了 提 高 自 己 的 能 力 ， 另 外 每 週 五 天 ， 請 你 好 好 地 學 習 自 己 有 興 趣 的 事 情 。
({'over_corr': 0, 'total_err': 2, 'true_

Epoch:35/35: 100%|██████████| 103/103 [00:26<00:00,  3.84it/s, loss=0.002]


二 十 世 紀 帶 來 很 多 創 新 進 社 會 應 用 。
二 十 世 紀 帶 來 很 多 創 新 及 社 會 應 用 。
二 十 世 紀 帶 來 很 多 創 新 及 社 會 應 用 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
我 還 不 知 道 如 果 這 件 事 是 每 一 個 裡 拜 四 跟 明 天 一 樣 ， 可 是 我 知 道 的 時 候 馬 上 告 訴 您 。
我 還 不 知 道 如 果 這 件 事 是 每 一 個 禮 拜 四 跟 明 天 一 樣 ， 可 是 我 知 道 的 時 候 馬 上 告 訴 您 。
我 還 不 知 道 如 果 這 件 事 是 每 一 個 禮 拜 四 跟 明 天 一 樣 ， 可 是 我 知 道 的 時 候 馬 上 告 訴 您 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
如 果 您 都 不 回 應 我 們 的 反 應 ， 我 們 就 會 把 您 告 警 擦 ， 或 者 鄰 長 。
如 果 您 都 不 回 應 我 們 的 反 應 ， 我 們 就 會 把 您 告 警 察 ， 或 者 鄰 長 。
如 果 您 都 不 回 應 我 們 的 反 應 ， 我 們 就 會 把 您 告 警 察 ， 或 者 鄰 長 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
美 一 個 晚 上 他 出 去 兩 點 以 前 再 家 。
每 一 個 晚 上 他 出 去 兩 點 以 前 再 回 家 。
每 一 個 晚 上 他 出 去 兩 點 以 前 再 回 家 。
({'over_corr': 0, 'total_err': 2, 'true_co

In [14]:
test(model, tokenizer, test_data_loader)

0.42000693965841224 {'over_corr': 2916, 'total_err': 5278, 'true_corr': tensor(1312, device='cuda:0')} {'over_corr': 1452, 'total_err': 3436, 'true_corr': 466}


In [15]:
123

123